In [16]:
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
from torch import nn
import numpy as np
import pandas as pd
import seaborn as sns
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import MinMaxScaler    
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
%matplotlib inline

In [22]:
columns = ['sex', 'length', 'diameter', 'height', 'whole_weight',
           'shucked_weight', 'viscera_weight', 'shell_weight', 'rings']
df = pd.read_csv('../data/abalone.csv', header=None, names=columns, index_col=False)

In [23]:
X = df.drop(columns = 'rings')
y = df.rings

In [24]:
X = pd.get_dummies(X)

In [25]:
# Train - Test
X_trainval, X_test, y_trainval, y_test = train_test_split(X, y, test_size=0.1, random_state=42)
# Split train into train-val
X_train, X_val, y_train, y_val = train_test_split(X_trainval, y_trainval, test_size=0.25, random_state=42)

In [26]:
X_train

,length,diameter,height,whole_weight,shucked_weight,viscera_weight,shell_weight,sex_F,sex_I,sex_M
350,0.610,0.495,0.185,1.1530,0.5360,0.2905,0.2450,1,0,0
928,0.435,0.340,0.115,0.3925,0.1825,0.0780,0.1145,0,1,0
1368,0.610,0.460,0.160,1.0000,0.4940,0.1970,0.2750,0,0,1
3108,0.375,0.285,0.080,0.2260,0.0975,0.0400,0.0725,0,1,0
1275,0.480,0.370,0.125,0.5435,0.2440,0.1010,0.1650,0,1,0
...,...,...,...,...,...,...,...,...,...,...
2002,0.360,0.270,0.085,0.1960,0.0905,0.0340,0.0530,0,1,0
1698,0.630,0.495,0.175,1.2695,0.6050,0.2710,0.3280,0,0,1
3792,0.655,0.520,0.175,1.4720,0.6275,0.2700,0.4500,1,0,0
2191,0.645,0.495,0.185,1.4935,0.5265,0.2785,0.4550,0,0,1


In [27]:
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)
X_train, y_train = np.array(X_train), np.array(y_train)
X_val, y_val = np.array(X_val), np.array(y_val)
X_test, y_test = np.array(X_test), np.array(y_test)

In [28]:
y_train, y_test, y_val = y_train.astype(float), y_test.astype(float), y_val.astype(float)

In [30]:
class RegressionDataset(Dataset):

    def __init__(self, X_data, y_data):
        self.X_data = X_data
        self.y_data = y_data

    def __getitem__(self, index):
        return self.X_data[index], self.y_data[index]

    def __len__ (self):
        return len(self.X_data)


train_dataset = RegressionDataset(torch.from_numpy(X_train).float(), torch.from_numpy(y_train).float())
val_dataset = RegressionDataset(torch.from_numpy(X_val).float(), torch.from_numpy(y_val).float())
test_dataset = RegressionDataset(torch.from_numpy(X_test).float(), torch.from_numpy(y_test).float())

In [38]:
EPOCHS = 150
BATCH_SIZE = 64
LEARNING_RATE = 0.1
NUM_FEATURES = len(X.columns)

In [32]:
train_loader = DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(dataset=val_dataset, batch_size=1)
test_loader = DataLoader(dataset=test_dataset, batch_size=1)

In [39]:
class MultipleRegression(nn.Module):
    def __init__(self, num_features):
        super(MultipleRegression, self).__init__()
        self.layer_1 = nn.Linear(num_features, 16)
        self.layer_2 = nn.Linear(16, 8)
        self.layer_out = nn.Linear(8, 1)
        self.relu = nn.ReLU()

    def forward(self, inputs):
        x = self.relu(self.layer_1(inputs))
        x = self.relu(self.layer_2(x))
        x = self.layer_out(x)
        return (x)

    def predict(self, test_inputs):
        x = self.relu(self.layer_1(test_inputs))
        x = self.relu(self.layer_2(x))
        x = self.layer_out(x)
        return (x)

In [40]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [41]:
model = MultipleRegression(NUM_FEATURES)
model.to(device)
print(model)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

MultipleRegression(
  (layer_1): Linear(in_features=10, out_features=16, bias=True)
  (layer_2): Linear(in_features=16, out_features=8, bias=True)
  (layer_out): Linear(in_features=8, out_features=1, bias=True)
  (relu): ReLU()
)


In [42]:
loss_stats = {
    'train': [],
    "val": []
}

In [43]:
print("Begin training.")
for e in (range(1, EPOCHS+1)):

    # TRAINING
    train_epoch_loss = 0
    model.train()
    for X_train_batch, y_train_batch in train_loader:
        X_train_batch, y_train_batch = X_train_batch.to(device), y_train_batch.to(device)
        optimizer.zero_grad()

        y_train_pred = model(X_train_batch)

        train_loss = criterion(y_train_pred, y_train_batch.unsqueeze(1))

        train_loss.backward()
        optimizer.step()

        train_epoch_loss += train_loss.item()


    # VALIDATION    
    with torch.no_grad():

        val_epoch_loss = 0

        model.eval()
        for X_val_batch, y_val_batch in val_loader:
            X_val_batch, y_val_batch = X_val_batch.to(device), y_val_batch.to(device)

            y_val_pred = model(X_val_batch)

            val_loss = criterion(y_val_pred, y_val_batch.unsqueeze(1))

            val_epoch_loss += val_loss.item()
            loss_stats['train'].append(train_epoch_loss/len(train_loader))
            loss_stats['val'].append(val_epoch_loss/len(val_loader))                              

    print(f'Epoch {e+0:03}: | Train Loss: {train_epoch_loss/len(train_loader):.5f} | Val Loss: {val_epoch_loss/len(val_loader):.5f}')

Begin training.
Epoch 001: | Train Loss: 17.55407 | Val Loss: 6.07335
Epoch 002: | Train Loss: 6.72857 | Val Loss: 5.67713
Epoch 003: | Train Loss: 6.44104 | Val Loss: 5.48693
Epoch 004: | Train Loss: 6.20568 | Val Loss: 5.01339
Epoch 005: | Train Loss: 5.41278 | Val Loss: 5.34052
Epoch 006: | Train Loss: 5.78001 | Val Loss: 7.02504
Epoch 007: | Train Loss: 5.69198 | Val Loss: 4.25778
Epoch 008: | Train Loss: 5.23727 | Val Loss: 6.27974
Epoch 009: | Train Loss: 5.73879 | Val Loss: 4.23330
Epoch 010: | Train Loss: 5.16105 | Val Loss: 4.45991
Epoch 011: | Train Loss: 5.30897 | Val Loss: 5.20341
Epoch 012: | Train Loss: 5.36370 | Val Loss: 4.69483
Epoch 013: | Train Loss: 5.34751 | Val Loss: 4.28200
Epoch 014: | Train Loss: 5.35688 | Val Loss: 7.10833
Epoch 015: | Train Loss: 7.12991 | Val Loss: 4.33333
Epoch 016: | Train Loss: 5.52922 | Val Loss: 4.27222
Epoch 017: | Train Loss: 5.27291 | Val Loss: 4.25436
Epoch 018: | Train Loss: 5.66321 | Val Loss: 4.87285
Epoch 019: | Train Loss: 6.21